All python imports are handled here

In [1]:
!pip install rpy2

In [2]:
!pip install rdata

In [3]:
!pip install matplotlib seaborn

In [4]:
!pip install cffi==1.12.3

In [12]:
import os
os.environ['R_HOME'] = 'C:\\Users\\riddh\\OneDrive - University of Toronto\\First Year\\Summer\\R-4.0.5'

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rpy2.robjects import r
import rpy2.robjects.pandas2ri as pandas2ri
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
#Import the SignatureTranslatedAnonymousPackage 
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage as STAP

#Importing a converter from R to pandas for data processing
from rpy2.robjects.conversion import localconverter

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/riddh/OneDrive - University of Toronto/First Year/Summer/R-4.0.5'

Working to install all necessary R packages

In [23]:
def rinit():
  # import rpy2's package module
  import rpy2.robjects.packages as rpackages

  # import R's utility package
  utils = rpackages.importr('utils')

  # select a mirror for R packages
  utils.chooseCRANmirror(ind=1) # select the first mirror in the list

  packnames = ('readxl', 'reshape2', 'tidyr', 'stringr')

  # R vector of strings
  from rpy2.robjects.vectors import StrVector

  # Selectively install what needs to be install.
  # We are fancy, just because we can.
  names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
  if len(names_to_install) > 0:
      utils.install_packages(StrVector(names_to_install))

rinit()

OSError: cannot load library 'C:\Users\riddh\ONEDRI~1\FIRSTY~1\Summer\R-40~1.5/lib/libR.so': C:\Users\riddh\ONEDRI~1\FIRSTY~1\Summer\R-40~1.5/lib/libR.so: cannot open shared object file: No such file or directory

Making the flame_us_code folder the current working directory

/content/drive/My Drive/flame_us_code


Verifying the current working folder: you should expect to see /content/drive/MyDrive/flame_us_code

/content/drive/My Drive/flame_us_code


Initializing the model

In [ ]:
def model_init():
  robjects.r.source("model_setup.R")
  robjects.r.source("load_functions.R")

model_init()

In [6]:
def FLAME_run_function(fun_name, index=0):
  filepath = 'functions/' + fun_name + '.R'
  var_to_open_as = fun_name.replace("_f", "")
  py_func_name = var_to_open_as + '_func'
    
  with open(filepath, 'r') as globals()[var_to_open_as]:
    globals()[py_func_name] = globals()[var_to_open_as].read()

  fun_res = STAP(globals()[py_func_name], fun_name + '.R') 
  cmd='fun_res.' + fun_name + '()'
  output = eval(cmd)
  if index:
    return(output[index])
  else:
    return output

print(FLAME_run_function(fun_name="fleet_vint_stock_f", index = None))

NameError: name 'STAP' is not defined

In [ ]:
with open('functions/survival_rate_f.R', 'r') as survival_rates:
  survival_rate_func = survival_rates.read()


fun_res = STAP(survival_rate_func, "survival_rate_f.R")
print(fun_res.__dict__)
fleet_vkt_res = fun_res.survival_rate_f(year=2015, size="Car")
print(fleet_vkt_res)

In [ ]:
print(type(fleet_vkt_res))

fleet_vkt_res_dt = pandas2ri.ri2py_vector(fleet_vkt_res)
print(type(fleet_vkt_res_dt))
print(fleet_vkt_res_dt)
colnames = fleet_vkt_res.colnames
rownames = fleet_vkt_res.rownames
print(colnames, rownames)
fleet_vkt_res_pd_dt = pd.DataFrame(fleet_vkt_res_dt, columns=colnames, index=rownames)
print(fleet_vkt_res_pd_dt)
plt.plot(colnames, fleet_vkt_res_dt[0])